<a href="https://colab.research.google.com/github/min02yam/project_2022_03/blob/main/H20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
install.packages("h2o")
library(h2o)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘bitops’, ‘RCurl’



----------------------------------------------------------------------

Your next step is to start H2O:
    > h2o.init()

For H2O package documentation, ask for help:
    > ??h2o

After starting H2O, you can use the Web UI at http://localhost:54321
For more information visit https://docs.h2o.ai

----------------------------------------------------------------------



Attaching package: ‘h2o’


The following objects are masked from ‘package:data.table’:

    hour, month, week, year


The following objects are masked from ‘package:stats’:

    cor, sd, var


The following objects are masked from ‘package:base’:

    &&, %*%, %in%, ||, apply, as.factor, as.numeric, colnames,
    colnames<-, ifelse, is.character, is.factor, is.numeric, log,
    log10, log1p, log2, round, signif, trunc




In [1]:
require(data.table)

Loading required package: googledrive



In [7]:
system("gdown --id 1PwFE91U4jhsO5gM4bWdnyvAClto3txOb")  # train
system("gdown --id 18elLaOJbV9rM-I0C-G1yswWKrc7ZV6yr")  # test

In [10]:
h2o.init(max_mem_size = "4g", nthreads = 5)


H2O is not running yet, starting it now...

Note:  In case of errors look at the following log files:
    /tmp/RtmpHBnS5M/file4c61d09c10/h2o_UnknownUser_started_from_r.out
    /tmp/RtmpHBnS5M/file4c5c2d3881/h2o_UnknownUser_started_from_r.err


Starting H2O JVM and connecting: ... Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         2 seconds 332 milliseconds 
    H2O cluster timezone:       Etc/UTC 
    H2O data parsing timezone:  UTC 
    H2O cluster version:        3.38.0.1 
    H2O cluster version age:    2 months and 17 days  
    H2O cluster name:           H2O_started_from_R_root_tpg424 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   4.00 GB 
    H2O cluster total cores:    2 
    H2O cluster allowed cores:  2 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    H2O Internal Security:      FALSE 
    R Version:    

In [12]:
DT= fread("train.csv")

In [13]:
h2o_dt = as.h2o(DT)

split_dt = h2o.splitFrame(h2o_dt, c(0.7), seed=49)
train =  h2o.assign(split_dt[[1]], "train")
test = h2o.assign(split_dt[[2]], "test")

  |======================================================================| 100%


In [14]:
y = "predicted_weight_g"
x = setdiff(names(train), y)

In [15]:
aml = h2o.automl(x = x, y = y,
                training_frame = train,
                max_models = 20,
                seed = 49)

  |======================================================================| 100%


In [16]:
lb = aml@leaderboard
print(lb, n = nrow(lb))

                                                  model_id      rmse       mse
1  StackedEnsemble_BestOfFamily_1_AutoML_1_20221207_115955  7.604948  57.83524
2     StackedEnsemble_AllModels_1_AutoML_1_20221207_115955  7.658306  58.64966
3                           GBM_4_AutoML_1_20221207_115955  7.876706  62.04250
4                           GBM_3_AutoML_1_20221207_115955  8.502536  72.29312
5                           GBM_5_AutoML_1_20221207_115955  8.795446  77.35987
6                           GBM_2_AutoML_1_20221207_115955  8.809905  77.61442
7          XGBoost_grid_1_AutoML_1_20221207_115955_model_2  8.914641  79.47083
8                       XGBoost_1_AutoML_1_20221207_115955  9.800329  96.04644
9                           XRT_1_AutoML_1_20221207_115955 10.181802 103.66909
10                      XGBoost_3_AutoML_1_20221207_115955 10.204073 104.12311
11         XGBoost_grid_1_AutoML_1_20221207_115955_model_1 10.497450 110.19646
12                      XGBoost_2_AutoML_1_20221207_

In [17]:
m = h2o.get_best_model(aml,criterion = "rmse")
h2o.performance(m, test) # AUC:  0.9983013

H2ORegressionMetrics: stackedensemble

MSE:  63.32925
RMSE:  7.957967
MAE:  5.103616
RMSLE:  NaN
Mean Residual Deviance :  63.32925


In [18]:
test = fread("test.csv")
h2o_test = as.h2o(test)

  |======================================================================| 100%


In [25]:
fit = h2o.predict(object=m, newdata = h2o_test) %>% 
  as.data.table()

  |======================================================================| 100%


In [22]:
fit_dt = as.data.table(fit)

In [26]:
fit %>% head

predict
<dbl>
0.2674741
2.7053392
0.6986194
2.0813989
2.6064324
4.2707098
